<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задача-№1" data-toc-modified-id="Задача-№1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задача №1</a></span></li><li><span><a href="#Задача-№2" data-toc-modified-id="Задача-№2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задача №2</a></span></li><li><span><a href="#Задача-№3" data-toc-modified-id="Задача-№3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задача №3</a></span></li><li><span><a href="#Задача-№4" data-toc-modified-id="Задача-№4-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Задача №4</a></span></li><li><span><a href="#Задача-№5" data-toc-modified-id="Задача-№5-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Задача №5</a></span></li></ul></div>

# Анализ базы данных сервиса для чтения книг по подписке

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Описание данных:**

Таблица `books`. Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`. Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `ratings`. Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`. Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**Цели и задачи:**
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Подключение к БД

Подключимся к БД, выведем первые 5 строк каждой таблицы.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
print('Таблица books')
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
print('Таблица authors')
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
print('Таблица ratings')
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [4]:
print('Таблица reviews')
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задача №1

Посчитайте, сколько книг вышло после 1 января 2000 года.

In [5]:
query = \
'''
SELECT COUNT(book_id) 
FROM books 
WHERE CAST(publication_date AS date) > '2000-01-01' 
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


**Вывод:** после 1 января 2000 года вышло 819 книг.

## Задача №2

Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [6]:
query = \
'''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT rev.review_id) as num_reviews,
       ROUND(AVG(rat.rating), 3) as avg_rating
FROM books AS b
LEFT JOIN ratings AS rat ON (b.book_id=rat.book_id)
LEFT JOIN reviews AS rev ON (b.book_id=rev.book_id)
GROUP BY b.book_id
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,num_reviews,avg_rating
0,1,'Salem's Lot,2,3.667
1,2,1 000 Places to See Before You Die,1,2.500
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.667
3,4,1491: New Revelations of the Americas Before C...,2,4.500
4,5,1776,4,4.000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.667
996,997,Xenocide (Ender's Saga #3),3,3.400
997,998,Year of Wonders,4,3.200
998,999,You Suck (A Love Story #2),2,4.500


**Вывод:** для каждой книги мы нашли количество обзоров и среднюю оценку.

## Задача №3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [37]:
query = \
'''
WITH books_leader AS
(SELECT publisher_id,
       COUNT(book_id) AS num_books
FROM books 
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY num_books DESC
LIMIT 1)
SELECT pub.publisher,
       bl.publisher_id,
       MAX(bl.num_books) AS max_books
FROM books_leader AS bl
JOIN publishers AS pub ON (bl.publisher_id=pub.publisher_id)
GROUP BY pub.publisher,
         bl.publisher_id
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,publisher_id,max_books
0,Penguin Books,212,42


**Вывод:** издательство *Penguin Books* имеет больше всех остальных издательств выпущенных книг, не включая брошюры.

## Задача №4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [10]:
# в подзапросе учитываем условие книги с 50 оценками и более
# в основном запросе считаем среднюю оценку по этим книгам и авторам
# фильтруем по убыванию и ограничиваем 1-й строкой

query = \
'''
WITH rating_books AS
(SELECT b.book_id,
       a.author
FROM books AS b
JOIN ratings AS rat ON (b.book_id=rat.book_id)
JOIN authors AS a ON (b.author_id=a.author_id)
GROUP BY b.book_id,
         a.author
HAVING COUNT(rat.rating_id)>=50)
SELECT rb.author,
       ROUND(AVG(rat.rating), 3) AS avg_rating
FROM ratings AS rat
JOIN rating_books AS rb ON (rat.book_id = rb.book_id)
GROUP BY rb.author
ORDER BY avg_rating DESC
LIMIT 1
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287


**Вывод:** автор с самой высокой оценкой среди книг, имеющих 50 оценок и более, J.K. Rowling/Mary GrandPré.

## Задача №5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [56]:
# в подзапросе учитываем условие пользователи поставившие больше 48 оценок
# и считаем количество их обзоров
# в основном запросе считаем среднее количество обзоров

query = \
'''
WITH num_rev_user AS
(SELECT user_rate.username,
       COUNT(rev.review_id) as num_rev
FROM (SELECT username
      FROM ratings 
      GROUP BY username
      HAVING COUNT(rating_id) > 48) AS user_rate
JOIN reviews AS rev ON (user_rate.username=rev.username)
GROUP BY user_rate.username)

SELECT AVG(num_rev) AS avg_num_rev
FROM num_rev_user
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_num_rev
0,24.0


**Вывод:** пользователи, которые поставили больше 48 оценок, в среднем оставляют по 24 обзора.

# Общий вывод

В ходе анализа базы данных сервиса для чтения книг по подписке были проведены следующие работы и получены следующие выводы:
- 819 книг было выпущено с 1 января 2000 года;
- по каждой книге были найдены количество обзоров и средняя оценка от пользователей;
- издательство Penguin Books имеет больше всех остальных издательств выпущенных книг, не включая брошюры (книги имеющие 50 или менее страниц);
- автор с самой высокой оценкой среди книг, имеющих 50 оценок и более, J.K. Rowling/Mary GrandPré, с средней оценкой 4.287;
- пользователи, которые поставили более 48 оценок оставляют в среднем по 24 обзора.